<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Preamble" data-toc-modified-id="Preamble-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Preamble</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Imports</a></span></li></ul></li><li><span><a href="#Testing" data-toc-modified-id="Testing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Testing</a></span></li></ul></div>

# Preamble

## Imports

In [173]:
import random as r
import pandas as pd

In [167]:
from data_structs import Person, Network

In [161]:
class Population():
    
    def __init__(self,id=None,population=None):
        self.people = pd.DataFrame(
            [{
                'per_id':i,
                'state':'sus',
                'infection_date': 'NaN',
                'death_date': 'NaN'
            } for i in range(0,population)]).set_index('per_id')
        self.id = id
        
    def add_person(self, state='sus',infection_date='NaN',death_date='NaN'):
        self.people = self.people.append(
            {
                'state':state,
                'infection_date': infection_date,
                'death_date': death_date
            }
            ,ignore_index=True)
        
    def update_person_state(self, per_id, state):
        self.people.iloc[per_id].state = state
        
    def infect_person(self, per_id, day):
        if isinstance(per_id,int):
            self.people.iloc[per_id]['infection_date','state'] = [day,'inf']
        else:
            for i in per_id:
                self.people.iloc[i]['infection_date','state'] = [day,'inf']
            
    def get_population_size(self):
        return len(self.people)
    
    def get_population(self):
        """ Returns the people dict of this population object.
        The people dict should have the following format:

            people = {person_id_int_1: Person_object_1,
                      .
                      .
                      .
                      person_id_int_n: Person_object_n
                      }
        """
        return self.people
    
    def count_infected(self):
        return self.people.state.value_counts()['inf']
    
    def count_states(self):
        return self.people.state.value_counts()

# Testing

In [162]:
cityville = Population(0,10)

In [163]:
cityville.add_person()

In [164]:
cityville.update_person_state(2,'gay')

In [165]:
cityville.infect_person([0,1],3)

In [166]:
cityville.count_states()

sus    8
inf    2
gay    1
Name: state, dtype: int64

In [198]:
network = Network(cityville)

In [199]:
network.network

,per_id,connections


In [178]:
pop_size = network.population.get_population_size()

In [181]:
network.population.people.index.min()

0

In [337]:
class Network():
    """ Network class which constructs a network for a population.

    This class allows writing out and reading in of network files, which track
    change of state for connections between persons within a population, allowing
    for simulations of networks which are temporally variant.

    Once a Network object is initialized, its network can be accessed with the
    Network.get_network() method. This method returns the network dict which
    contains all the connection lists.
    """
    def __init__(self, population,conn_min=0,conn_max=5):
        self.population = population
        self.network = self.population.people.filter('index')
        self.network['connections'] = None
        self.conn_min = conn_min
        self.conn_max = conn_max
        
    def _get_connections(self,connections):
        num_connections = r.randint(self.conn_min,self.conn_max)
        conn_min = self.population.people.index.min()
        conn_max = self.population.people.index.max()
        connections = []
        for i in range(0, num_connections):
            conn_id = r.randint(conn_min,conn_max)
            while conn_id in connections:
                conn_id = r.randint(conn_min,conn_max)
            connections.append(conn_id)
        return connections
    
    def get_connections(self):
        self.network.connections = self.network.connections.apply(self._get_connections)

In [347]:
network = Network(Population(id=0,population=1000000))

In [ ]:
%%timeit
network.get_connections()

In [344]:
from data_structs import Network as Network_orig
from data_structs import Population as Population_orig

In [345]:
network_orig = Network_orig(Population_orig(0,1000000))

In [346]:
%%timeit
network_orig.init_random_network(0,5,23)

88.5 ms ± 29.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [ ]:
def init_random_network(self, connection_min, connection_max, seed_num, verbose=False):
    r.seed(seed_num)
    pop = self.population.get_population()
    pop_size =len(pop)

    completion_percent = 0
    for person_id in pop:
        num_connections = r.randint(connection_min, connection_max)
        connections_list = []
        for i in range(0, num_connections):
            # Get a random person_id
            connection_id = r.randint(0, pop_size-1)
            # Ensure it isn't in the connections_list
            while(connection_id in connections_list):
                connection_id = r.randint(0, pop_size-1)
            # Add the random person_id to the connections_list
            connections_list.append(connection_id)
            completion_percent = (person_id / pop_size) * 100
            if(verbose):
                print("Generating random network: " + str(completion_percent) + "%")
        # Add the connections list to the network dict
        self.network[person_id] = connections_list

In [171]:
connection_min = 0
connection_max = 5

In [175]:
r.randint(connection_min, connection_max)

4

In [186]:
pd.DataFrame.from_dict(network.network,columns=['per_id','connections'])

ValueError: cannot use columns parameter with orient='columns'